test codes to see if i can just get timestamps and export it to cvs file fot easier handling

In [1]:
import pandas as pd
import gzip
import json

file_path = 'C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/1653862941-dcfb4c8f-dc3e-4247-a564-45e1ab536dac.jsonl.gz'

# Extract data from the file
data = []
with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    for line in file:
        json_obj = json.loads(line)
        if json_obj.get('event_type') == 'combat_start':
            data.append({'timestamp': json_obj.get('timestamp')})

# Convert the extracted data to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Export to CSV
csv_file_path = 'C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/combat_start_timestamps.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data exported to CSV file at: {csv_file_path}")


      timestamp
0  1.653863e+09
Data exported to CSV file at: C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/combat_start_timestamps.csv


test code to see if i can also get combat id, for the combat id to get combat start timestamp, number of player combatants and number of monster combatants. Also gets character names and monster names too.

In [4]:
import pandas as pd
import gzip
import json
from collections import defaultdict

def process_file(file_path):
    # Dictionary to hold combat start times and actor details
    combat_data = defaultdict(lambda: {'start_time': None, 'player_ids': set(), 'player_names': set(),
        'monster_ids': set(), 'monster_names': set()})

    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            event = json.loads(line)
            combat_id = event.get("combat_id")

            # Check for combat start event
            if event.get("event_type") == "combat_start":
                combat_data[combat_id]['start_time'] = event.get("timestamp")

            # Check for player actor join events
            elif event.get("event_type") == "command" and event.get("command_name") == "init join":
                player_id = event.get("author_id")
                player_name = event.get("caster", {}).get("name", event.get("author_name")) # Default to author_name if caster name is not present
                combat_data[combat_id]['player_ids'].add(player_id)
                combat_data[combat_id]['player_names'].add(player_name)

            # Check for monster actor events
            elif event.get("event_type") == "combat_state_update":
                combat = event.get("data")
                for actor in combat.get("combatants", []):
                    if actor["type"] == "group":
                        for group_actor in actor["combatants"]:
                            if group_actor["type"] == "monster":
                                combat_data[combat_id]['monster_ids'].add(group_actor["id"])
                                combat_data[combat_id]['monster_names'].add(group_actor["name"])
                    elif actor["type"] == "monster":
                        combat_data[combat_id]['monster_ids'].add(actor["id"])
                        combat_data[combat_id]['monster_names'].add(actor["name"])

    # Convert to a more structured format
    processed_data = []
    for cid, data in combat_data.items():
        processed_data.append({
            "combat_id": cid,
            "start_time": data['start_time'],
            "num_player_actors": len(data['player_ids']),
            "player_ids": list(data['player_ids']),
            "player_names": list(data['player_names']),
            "num_monster_actors": len(data['monster_ids']),
            "monster_ids": list(data['monster_ids']),
            "monster_names": list(data['monster_names'])
        })

    return processed_data

# Define file path

file_path = 'C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/1669628280-3853b404-f342-41b6-ab08-02389ab64493.jsonl.gz'
# Process the file
combat_info = process_file(file_path)

# Convert to DataFrame
df = pd.DataFrame(combat_info)

# Export to CSV
csv_file_path = 'C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/combat_analysistest.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data exported to CSV file at: {csv_file_path}")


Data exported to CSV file at: C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/combat_analysistest.csv


test if i can get hp out of human readable string 

In [56]:
import re

# Example string from the "human_readable" section
human_readable = "... 22: Ellianna Elara Esmeray-Heisenberg <26/184 HP> ..."

# Regex pattern to match the player names and HP ratios
pattern = r'\d+: ([\w\s.-]+) <(\d+/\d+ HP)>'

# Find all matches
matches = re.findall(pattern, human_readable)

# Extract info like hp with the name
extracted_info = {}
for match in matches:
    name, hp_ratio = match
    extracted_info[name] = hp_ratio

# Display the extracted info
print(extracted_info)


{'Ellianna Elara Esmeray-Heisenberg': '26/184 HP'}


test if I can get human_readable part and save the hp associated with player_name. and HP ratio!

In [1]:
import pandas as pd
import gzip
import json
import re
from collections import defaultdict

def process_file(file_path):
    combat_data = defaultdict(lambda: {
        'start_time': None, 
        'player_ids': set(),
        'player_info': {},
        'monster_ids': set(), 
        'monster_names': set()
    })
    last_human_readable = {}

    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        for line in file:
            event = json.loads(line)
            combat_id = event.get("combat_id")

            # Checking for combat start and getting timestamp (dont think we actually need timestamp)
            if event.get("event_type") == "combat_start":
                combat_data[combat_id]['start_time'] = event.get("timestamp")

            # counting players who joined initiative
            elif event.get("event_type") == "command" and event.get("command_name") == "init join":
                player_id = event.get("author_id")
                player_name = event.get("caster", {}).get("name", event.get("author_name"))
                combat_data[combat_id]['player_ids'].add(player_id)
                combat_data[combat_id]['player_info'][player_name] = {'hp_ratio': None}

            # counting monsters
            elif event.get("event_type") == "combat_state_update":
                combat = event.get("data")
                for actor in combat.get("combatants", []):
                    if actor["type"] == "monster":
                        combat_data[combat_id]['monster_ids'].add(actor["id"])
                        combat_data[combat_id]['monster_names'].add(actor["name"])
                
                #trying to get that human readable part
                human_readable = event.get("human_readable", "")
                if human_readable:
                    last_human_readable[combat_id] = human_readable

    # Processing the last human_readable to extract HP ratios (this is not working ....)
    for cid, human_readable in last_human_readable.items():
        pattern = r'\d+: ([\w\s.-]+) <(\d+/\d+ HP)>'
        matches = re.findall(pattern, human_readable)

        for name, hp_ratio in matches:
            if name in combat_data[cid]['player_info']:
                combat_data[cid]['player_info'][name]['hp_ratio'] = hp_ratio

    # Printing the last human_readable string for each combat ID
    for cid, human_readable in last_human_readable.items():
        print(f"Last 'human_readable' for {cid}: {human_readable}")

    # Creating the DataFrame from collected data
    processed_data = []
    for cid, data in combat_data.items():
        processed_data.append({
            "combat_id": cid,
            "start_time": data['start_time'],
            "num_player_actors": len(data['player_ids']),
            "player_info": data['player_info'],
            "num_monster_actors": len(data['monster_ids']),
            "monster_ids": list(data['monster_ids']),
            "monster_names": list(data['monster_names']),
        })

    return pd.DataFrame(processed_data)

# Define file path
file_path = 'C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/1669628280-3853b404-f342-41b6-ab08-02389ab64493.jsonl.gz'

# Process the file and create DataFrame
df = process_file(file_path)

# Export to CSV
csv_file_path = 'C:/Users/josep/OneDrive/Desktop/Erdos/anonymized/data/combat_analysis_with_hp.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data exported to CSV file at: {csv_file_path}")



Last 'human_readable' for 1669628280-3853b404-f342-41b6-ab08-02389ab64493: ```md
Current initiative: 31 (round 8)
  37: Merizon Aerialis <1/314 HP> (AC 45, Hasted [4 rounds], Hero.3 [3 rounds], Envelops with intense love, Enveloped by love, Aura of Protection, Aura of Warding, Total Cover, Paladin Supreme, Charismatic, Juro's Ancient Protector, Blessed [9 rounds], All-Seeing [10 rounds])
  36: Talia Draculesti <70/200 HP> (AC 40, Death Ward [8.0 hours], Conjure Animals [59.3 minutes], Envelops with intense love, Enveloped by love, Aura of Protection, Aura of Warding, Total Cover, Paladin Supreme, Charismatic, Juro's Ancient Protector)
# 31: Dragahree <384/407 HP> (AC 39, Envelops with intense love, Enveloped by love, Prone, Aura of Protection, Aura of Warding, Total Cover, Paladin Supreme, Charismatic, Juro's Ancient Protector)
  22: Ellianna Elara Esmeray-Heisenberg <26/184 HP> (AC 31, Death Ward [8.0 hours], Envelops with intense love, Enveloped by love, Trance of Order [5 rounds], A